In [1]:
import pandas as pd
import altair as alt
import unidecode
import geobr
from datetime import date, timedelta

In [2]:
%load_ext pycodestyle_magic
%flake8_on

In [3]:
# This depends on  the time the script is run.

recent = "2020-05-16"

In [4]:
def decodar(coluna):
    decode = [unidecode.unidecode(m).upper() for m in coluna]
    return decode


In [5]:
# Depends on file generated by the "get_csv.py" on other folder

brasil = pd.read_excel(f"../data-by-state/{recent}.xlsx")
brasil = brasil[brasil['data'] == recent]
brasil.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos
80,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-05-16,20,210147125.0,233142,15633,89672.0,127837.0
127,Centro-Oeste,MS,Alcinópolis,50,500025.0,50001.0,Campo Grande,2020-05-16,20,5343.0,1,0,NaN,NaN
134,Centro-Oeste,MS,Amambai,50,500060.0,50003.0,Dourados,2020-05-16,20,39396.0,4,0,NaN,NaN
161,Centro-Oeste,MS,Bataguassu,50,500190.0,50004.0,Três Lagoas,2020-05-16,20,23024.0,3,0,NaN,NaN
211,Centro-Oeste,MS,Batayporã,50,500200.0,50003.0,Dourados,2020-05-16,20,11329.0,6,2,NaN,NaN


In [6]:
estados = list(brasil['estado'].unique())
del estados[0]

In [7]:
municipios = brasil[~brasil['municipio'].isnull()]
municipios.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos
127,Centro-Oeste,MS,Alcinópolis,50,500025.0,50001.0,Campo Grande,2020-05-16,20,5343.0,1,0,NaN,NaN
134,Centro-Oeste,MS,Amambai,50,500060.0,50003.0,Dourados,2020-05-16,20,39396.0,4,0,NaN,NaN
161,Centro-Oeste,MS,Bataguassu,50,500190.0,50004.0,Três Lagoas,2020-05-16,20,23024.0,3,0,NaN,NaN
211,Centro-Oeste,MS,Batayporã,50,500200.0,50003.0,Dourados,2020-05-16,20,11329.0,6,2,NaN,NaN
220,Centro-Oeste,MS,Bela Vista,50,500210.0,50001.0,Campo Grande,2020-05-16,20,24629.0,2,0,NaN,NaN


## Example: Rio Grande do Sul

In [8]:
RS = municipios.query("estado=='RS'")
RS.loc[:, 'municipio'] = decodar(RS.loc[:, 'municipio'])

/home/lubianat/.local/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
mapa = geobr.read_municipality(code_muni='RS', year=2018)
mapa.rename(columns={'name_muni': 'municipio'}, inplace=True)
mapa.loc[:, 'municipio'] = decodar(mapa.loc[:, 'municipio'])

In [10]:
full = pd.merge(mapa, RS, on='municipio',  how='left')

In [11]:
full["casosAcumulado"] = full["casosAcumulado"].fillna(0)

natural_order = ["0",
                 "1-10",
                 "11-50",
                 "51-500",
                 ">500",
                 ">1000",
                 ">10000"]

full["casos_categorizados"] = pd.cut(full["casosAcumulado"],
                                     bins=[-1, 1, 10,
                                     50, 500, 1000,
                                     10000, 100000000],
                                     labels=natural_order)

Instructions on how to add save function to altair:
* https://github.com/altair-viz/altair_saver#nodejs - Had to npm install vega too
* https://github.com/altair-viz/altair_saver/issues/13

In [12]:
alt.Chart(full).mark_geoshape(
    stroke="lightgray",
    strokeOpacity=0.2
).encode(
    alt.Color('casos_categorizados:O',
              sort=natural_order,
              scale=alt.Scale(scheme='lightorange'))
).properties(
    width=450,
    height=300
).configure_legend(
    title=None
).configure_view(
    strokeWidth=0
).save(f"RS_{recent}.svg", method='node')

## Now running for all states
This takes some time (though not as much as I had imagined)

I'm saving maps as svg, but pngs are also an option, [some discussion about that here](https://en.wikipedia.org/wiki/Wikipedia:Blank_maps)

In [25]:
def get_estado(municipios, est):

    estado = municipios.query("estado==@est")
    estado.loc[:, 'municipio'] = decodar(estado.loc[:, 'municipio'])

    return(estado)


def get_mapa(est):

    geobr.read_municipality(code_muni=f"{est}", year=2018)
    mapa.rename(columns={'name_muni': 'municipio'}, inplace=True)
    mapa.loc[:, 'municipio'] = decodar(mapa.loc[:, 'municipio'])

    return(mapa)


def get_tabela_completa(mapa, estado, natural_order):

    full = pd.merge(mapa, estado, on='municipio',  how='left')
    full["casosAcumulado"] = full["casosAcumulado"].fillna(0)
    full["casos_categorizados"] = pd.cut(full["casosAcumulado"],
                                         bins=[-1, 1, 10,
                                         50, 500, 1000,
                                         10000, 100000000],
                                         labels=natural_order)
    return(full)


def get_altair_map(full, natural_order):
    mapa_para_retornar = alt.Chart(full).mark_geoshape(
        stroke="lightgray",
        strokeOpacity=0.2
    ).encode(
        alt.Color('casos_categorizados:O',
                  sort=natural_order,
                  scale=alt.Scale(scheme='lightorange'))
    ).properties(
        width=450,
        height=300
    ).configure_legend(
        title=None
    ).configure_view(
        strokeWidth=0
    )
    return(mapa_para_retornar)

import datetime
from datetime import date, timedelta

def craft_description(place, data_dos_dados):
    source_url = "https://covid.saude.gov.br/"
    today = date.today()
    today_in_commons_format = today.strftime("%Y-%m-%d")
    
    result = '''=={{int:filedesc}}==
{{Information
|description={{pt|1=Casos de COVID-19 por município no estado de ''' + place + " até o dia " + data_dos_dados + ''' de 2020.
Gráfico em Python gerado a partir dos dados de [''' + source_url + '''].
Script disponível em github.com/lubianat/wikidata_covid19/tree/master/sandbox/distribution_maps}}.
|date=''' + today_in_commons_format + '''
|source={{own}}
|author=[[User:TiagoLubiana|TiagoLubiana]] and [[User:Jvcavv]]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{self|cc-by-sa-4.0}}

[[Category:COVID-19_pandemic_in_Brazil_by_state]]
'''

    return(result)


48:1: E305 expected 2 blank lines after class or function definition, found 1
48:1: E402 module level import not at top of file
49:1: E402 module level import not at top of file
51:1: E302 expected 2 blank lines, found 1
55:1: W293 blank line contains whitespace
60:80: E501 line too long (98 > 79 characters)


In [35]:
for est in estados:
    estado = get_estado(municipios, est)
    mapa = get_mapa(est)

    natural_order = ["0",
                     "1-10",
                     "11-50",
                     "51-500",
                     ">500",
                     ">1000",
                     ">10000"]
    
    full = get_tabela_completa(mapa, estado, natural_order)
    mapa_plotado_via_altair = get_altair_map(full, natural_order)
    
    nome_do_arquivo_da_figura = f"./fig/{est}_{recent}.svg"
    mapa_plotado_via_altair.save(nome_do_arquivo_da_figura, method='node')
    print(mapa_plotado_via_altair)
    break

12:1: W293 blank line contains whitespace
15:1: W293 blank line contains whitespace


alt.Chart(...)


In [ ]:
data_dos_dados="16 de maio de 2020"

description = craft_description(place=est, data_dos_dados=data_dos_dados)

data_dos_dados_sem_espaco = "_".join(data_dos_dados.split(" "))
nome_do_arquivo_descrevendo_a_figura = "fig/description_cases_" + est + "_" + data_dos_dados_sem_espaco + ".txt"

file2 = open(nome_do_arquivo_descrevendo_casos, "w")
file2.write(description)
file2.close()


command_cases = "yes N | python3 upload.py -keep -filename " + nome_do_arquivo_da_figura + ' -summary:"updating status for today"' + " $(cat " + nome_do_arquivo_descrevendo_a_figura + ")"
print(command_cases)
! $command_cases

yes N | python3 upload.py -keep -filename ./fig/MS_2020-05-16.svg -summary:"updating status for today" $(cat fig/description_cases_MS_16_de_maio_de_2020.txt)
/home/lubianat/.local/lib/python3.6/site-packages/pywikibot/config2.py:1091: UserWarning: 
Configuration variable "use_api_login" is defined in your user-
config.py but unknown. It can be a misspelled one or a variable that
is no longer supported.
  'supported.'.format(name)), UserWarning)
['./fig/MS_2020-05-16.svg']
Password for user TiagoLubiana on commons:commons (no characters will be shown): 